This notebook presents the code for training the EfficientNetB7 model on Google TPU accelerator with various augmentations and visualization of the main steps of data processing and model training. 

- This work is based on two published notebooks, so please, upvote their:
    - pipeline was taken from @maksymshkliarevskyi notebook;
    - Needle augmentation was taken from @khoongweihao notebook

<a id="start"></a>
<h2 style='color:#0A0502; background:white; border:2px solid #0A0502'><center>Table of contents:</center></h2>

* [**Data preprocessing**](#1)
* [**Augmentations**](#2)
* [**Model preparation**](#3)
* [**Model training**](#4)
* [**Visualization of CNN intermediate activations**](#5)
* [**Prediction**](#6)

In [ ]:
# install the latest version of pip installer
!pip install --upgrade pip

In [ ]:
# install library with efficientnet architecture
!pip install -q efficientnet

In [ ]:
import re,os,cv2,random

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import albumentations as A
from sklearn.model_selection import train_test_split

# importing neural network architecture
from efficientnet.tfkeras import EfficientNetB7

# importing other useful tools: layers, optimizers, loss functions
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

%matplotlib inline 
print("Tensorflow version " + tf.__version__)

#### What is a TPU?

To accelerate the largest-scale machine learning (ML) applications deployed today and enable rapid development of the ML applications of tomorrow, Google created custom silicon chips called Tensor Processing Units ([TPUs](https://cloud.google.com/tpu/docs/tpus)). When assembled into multi-rack ML supercomputers called Cloud TPU Pods.


<center><img src="https://miro.medium.com/max/890/1*16HkeV33jzWruFoVYokVlQ.png" width="400"></center>
<br>


#### What’s in a Cloud TPU
A single Cloud TPU Pod can include more than 1,000 individual TPU chips which are connected by an ultra-fast, two-dimensional toroidal mesh network, as illustrated below. The TPU software stack uses this mesh network to enable many racks of machines to be programmed as a single, giant ML supercomputer via a variety of flexible, high-level APIs.




References:
- [Use TPUs](https://www.tensorflow.org/guide/tpu)
- [Better performance with the tf.data API](https://www.tensorflow.org/guide/data_performance)
- [Custom training with tf.distribute.Strategy](https://www.tensorflow.org/tutorials/distribute/custom_training)
- [Cloud TPU](https://cloud.google.com/tpu)
- [Google’s scalable supercomputers for machine learning, Cloud TPU Pods, are now publicly available in beta](https://cloud.google.com/blog/products/ai-machine-learning/googles-scalable-supercomputers-for-machine-learning-cloud-tpu-pods-are-now-publicly-available-in-beta)

In [ ]:
# TPU or GPU detection
# detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU: {tpu.master()}')
except ValueError:
    tpu = None
    print('Running on GPU')

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

REPLICAS = strategy.num_replicas_in_sync
print(f'Number of replicas: {REPLICAS}')

<a id="1"></a>
<h1 style='color:#0A0502; background:white; border:0'><center>Data preprocessing</center></h1>

[**Back to the table of contents**](#start)

In [ ]:
WORK_DIR = '../input/ranzcr-clip-catheter-line-classification'
os.listdir(WORK_DIR)

In [ ]:
# data connection
GCS_DS_PATH = KaggleDatasets().get_gcs_path('ranzcr-clip-catheter-line-classification')

train = pd.read_csv(os.path.join(WORK_DIR, "train.csv"))
train_images = GCS_DS_PATH + "/train/" + train['StudyInstanceUID'] + '.jpg'

ss = pd.read_csv(os.path.join(WORK_DIR, 'sample_submission.csv'))
test_images = GCS_DS_PATH + "/test/" + ss['StudyInstanceUID'] + '.jpg'

label_cols = ss.columns[1:]
labels = train[label_cols].values

train_annot = pd.read_csv(os.path.join(WORK_DIR, "train_annotations.csv"))

train.head()

In [ ]:
# show label classes
sns.set_style("whitegrid")
fig = plt.figure(figsize = (15, 12), dpi = 600)
plt.suptitle('Labels count', fontfamily = 'serif', size = 20)

for ind, i in enumerate(label_cols):
    fig.add_subplot(4, 3, ind + 1)

    sns.countplot(train[i], edgecolor = 'black',
                  palette = reversed(sns.color_palette('viridis', 2)))
    
    plt.xlabel('')
    plt.ylabel('')
    plt.xticks(fontfamily = 'serif', size = 12)
    plt.yticks(fontfamily = 'serif', size = 12)
    plt.title(i, fontfamily = 'serif', size = 12)
plt.show()

In [ ]:
# show some images
sample = train.sample(9)
plt.figure(figsize=(10, 7), dpi = 600)
for ind, image_id in enumerate(sample.StudyInstanceUID):
    plt.subplot(3, 3, ind + 1)
    image = image_id + '.jpg'
    img = cv2.imread(os.path.join(WORK_DIR, "train", image))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.title('Shape: {}'.format(img.shape[:2]))
    plt.axis("off")
plt.show()

In [ ]:
row = train_annot.iloc[8]
image_path = os.path.join(WORK_DIR, "train", row["StudyInstanceUID"] + ".jpg")
chosen_image = cv2.imread(image_path)

<a id="2"></a>
<h1 style='color:#0A0502; background:white; border:0'><center>Augmentations</center></h1>

[**Back to the table of contents**](#start)

### Albumentations library

Let's see what our training data would look like using the Albumentations library.

In [ ]:
albumentation_list = [A.RandomSunFlare(p=1), 
                      A.RandomFog(p=1), 
                      A.RandomBrightness(p=1),
                      A.RandomCrop(p=1,height = 512, width = 512), 
                      A.Rotate(p=1, limit=90),
                      A.RGBShift(p=1), 
                      A.RandomSnow(p=1),
                      A.HorizontalFlip(p=1), 
                      A.VerticalFlip(p=1), 
                      A.RandomContrast(limit = 0.5,p = 1),
                      A.HueSaturationValue(p=1,hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=50),
                      A.Cutout(p=1),
                      A.Transpose(p=1), 
                      A.JpegCompression(p=1),
                      A.CoarseDropout(p=1),
                      A.IAAAdditiveGaussianNoise(loc=0, scale=(2.5500000000000003, 12.75), per_channel=False, p=1),
                      A.IAAAffine(scale=1.0, translate_percent=None, translate_px=None, rotate=0.0, shear=0.0, order=1, cval=0, mode='reflect', p=1),
                      A.IAAAffine(rotate=90., p=1),
                      A.IAAAffine(rotate=180., p=1)]

In [ ]:
img_matrix_list = []
bboxes_list = []
for aug_type in albumentation_list:
    img = aug_type(image = chosen_image)['image']
    img_matrix_list.append(img)

img_matrix_list.insert(0,chosen_image)    

titles_list = ["Original","RandomSunFlare","RandomFog","RandomBrightness",
               "RandomCrop","Rotate", "RGBShift", "RandomSnow","HorizontalFlip", "VerticalFlip", "RandomContrast","HSV",
               "Cutout","Transpose","JpegCompression","CoarseDropout","IAAAdditiveGaussianNoise","IAAAffine","IAAAffineRotate90","IAAAffineRotate180"]

def plot_multiple_img(img_matrix_list, title_list, ncols, nrows=5,  main_title=""):
    fig, myaxes = plt.subplots(figsize=(20, 15), nrows=nrows, ncols=ncols, squeeze=False)
    fig.suptitle(main_title, fontsize = 30)
    fig.subplots_adjust(wspace=0.3)
    fig.subplots_adjust(hspace=0.3)
    for i, (img, title) in enumerate(zip(img_matrix_list, title_list)):
        myaxes[i // ncols][i % ncols].imshow(img)
        myaxes[i // ncols][i % ncols].set_title(title, fontsize=15)
    plt.show()
    
plot_multiple_img(img_matrix_list, titles_list, ncols = 4,main_title="Different Types of Augmentations with Albumentations")

### Xray Needle Augmentation

In [ ]:
def NeedleAugmentation(image, n_needles=2, dark_needles=False, p=0.5, needle_folder='../input/xray-needle-augmentation'):
    aug_prob = random.random()
    if aug_prob < p:
        height, width, _ = image.shape  # target image width and height
        needle_images = [im for im in os.listdir(needle_folder) if 'png' in im]

        for _ in range(1, n_needles):
            needle = cv2.cvtColor(cv2.imread(os.path.join(needle_folder, random.choice(needle_images))), cv2.COLOR_BGR2RGB)
            needle = cv2.flip(needle, random.choice([-1, 0, 1]))
            needle = cv2.rotate(needle, random.choice([0, 1, 2]))

            h_height, h_width, _ = needle.shape  # needle image width and height
            roi_ho = random.randint(0, abs(image.shape[0] - needle.shape[0]))
            roi_wo = random.randint(0, abs(image.shape[1] - needle.shape[1]))
            roi = image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

            # Creating a mask and inverse mask 
            img2gray = cv2.cvtColor(needle, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)

            # Now black-out the area of needle in ROI
            img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

            # Take only region of insect from insect image.
            if dark_needles:
                img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
                needle_fg = cv2.bitwise_and(img_bg, img_bg, mask=mask)
            else:
                needle_fg = cv2.bitwise_and(needle, needle, mask=mask)

            # Put needle in ROI and modify the target image
            dst = cv2.add(img_bg, needle_fg, dtype=cv2.CV_64F)

            image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst

    return image

In [ ]:
# orginal needles
chosen_image = cv2.imread(image_path)
aug_image = NeedleAugmentation(chosen_image, n_needles=3, dark_needles=False, p=1.0)
plt.imshow(aug_image)

In [ ]:
# dark needles (just black, dark, sinister as it is on an Xray)
chosen_image = cv2.imread(image_path)
aug_image = NeedleAugmentation(chosen_image, n_needles=3, dark_needles=True, p=1.0)
plt.imshow(aug_image)

### TensorFlow augmentations

In [ ]:
chosen_image = cv2.imread(image_path)

tf_trans_list = [
    tf.image.rot90(chosen_image, k=1), # 90 degrees counter-clockwise
    tf.image.rot90(chosen_image, k=2), # 180 degrees counter-clockwise
    tf.image.rot90(chosen_image, k=3), # 270 degrees counter-clockwise
    tf.image.random_brightness(chosen_image, 0.5), 
    tf.image.random_contrast(chosen_image, 0.2, 0.5), 
    tf.image.random_flip_left_right(chosen_image, seed=42),
    tf.image.random_flip_up_down(chosen_image, seed=42),
    tf.image.random_hue(chosen_image, 0.5),
    tf.image.random_jpeg_quality(chosen_image, 35, 50), 
    tf.image.random_saturation(chosen_image, 5, 10), 
    tf.image.transpose(chosen_image),
]

In [ ]:
img_matrix_list = []
bboxes_list = []
for aug_image in tf_trans_list:
    img_matrix_list.append(aug_image)

img_matrix_list.insert(0, chosen_image)    

titles_list = ["Original","Rotate90","Rotate180","Rotate270","RandomBrightness","RandomContrast","RandomLeftRightFlip","RandomUpDownFlip",
               "RandomHue","RandomJPEGQuality","RandomSaturation","Transpose"]

plot_multiple_img(img_matrix_list, titles_list, ncols = 3, nrows=4, main_title="Different Types of Augmentations with TensorFlow")

<a id="3"></a>
<h1 style='color:#0A0502; background:white; border:0'><center>Model preparation</center></h1>

[**Back to the table of contents**](#start)

In [ ]:
# main parameters
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16 * REPLICAS
STEPS_PER_EPOCH = len(train) * 0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train) * 0.2 / BATCH_SIZE
EPOCHS = 30
TARGET_SIZE = 600

In [ ]:
def build_decoder(with_labels = True,
                  target_size = (TARGET_SIZE, TARGET_SIZE), 
                  ext = 'jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels = 3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels = 3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode

# in this part you can choose any type of augmentation from the ones suggested above
def build_augmenter(with_labels = True):
    def augment(img):
        #img = NeedleAugmentation(img, n_needles=2, dark_needles=False, p=0.5)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_brightness(img, 0.9, 1)
        img = tf.image.random_contrast(img, 0.9, 1)
        #img = tf.image.random_saturation(img, 0.9, 1) 
        
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels = None, bsize = 32, cache = True,
                  decode_fn = None, augment_fn = None,
                  augment = True, repeat = True, shuffle = 1024, 
                  cache_dir = ""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls = AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls = AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [ ]:
# train test split
(train_img, valid_img, 
 train_labels, valid_labels) = train_test_split(train_images, labels, 
                                                train_size = 0.8, 
                                                random_state = 42)

In [ ]:
# Tensorflow datasets
train_df = build_dataset(
    train_img, train_labels, bsize = BATCH_SIZE, 
    cache = True)

valid_df = build_dataset(
    valid_img, valid_labels, bsize = BATCH_SIZE, 
    repeat = False, shuffle = False, augment = False, 
    cache = True)

In [ ]:
def create_model(model):
    conv_base = model(include_top = False, weights = 'imagenet',
                         input_shape = (TARGET_SIZE, TARGET_SIZE, 3))
    model = conv_base.output
    model = layers.GlobalAveragePooling2D()(model)
    model = layers.Dense(11, activation = "sigmoid")(model)
    model = models.Model(conv_base.input, model)

    model.compile(optimizer = 'adam',
                  loss = "binary_crossentropy",
                  metrics = [tf.keras.metrics.AUC(multi_label = True)])
    return model

In [ ]:
with strategy.scope():
    model_EfficientNetB7 = create_model(EfficientNetB7) 

In [ ]:
#tf.keras.utils.plot_model(model_EfficientNetB7, show_shapes=False)
print('EfficientNetB7 CNN has %d layers' %len(model_EfficientNetB7.layers))

In [ ]:
model_save_EfficientNetB7 = ModelCheckpoint('./EfficientNetB7_best_weights_TPU.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_auc', 
                             mode = 'max', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_auc', min_delta = 0.0001, 
                           patience = 5, mode = 'max', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_auc', patience = 3, min_lr=1e-6, 
                              mode = 'max', verbose = 1)

<a id="4"></a>
<h1 style='color:#0A0502; background:white; border:0'><center>Model training</center></h1>

[**Back to the table of contents**](#start)

In [ ]:
history_EfficientNetB7 = model_EfficientNetB7.fit(
    train_df,
    epochs = EPOCHS,
    steps_per_epoch = STEPS_PER_EPOCH,
    validation_data = valid_df,
    validation_steps = VALIDATION_STEPS,
    callbacks = [model_save_EfficientNetB7, early_stop, reduce_lr]
)

In [ ]:
auc = history_EfficientNetB7.history['auc']
val_auc = history_EfficientNetB7.history['val_auc']
loss = history_EfficientNetB7.history['loss']
val_loss = history_EfficientNetB7.history['val_loss']

epochs = range(1, len(auc) + 1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
sns.set_style("white")
plt.suptitle('Train history for EfficientNetB7 model', size = 15)

ax1.plot(epochs, auc, "bo", label = "Training auc")
ax1.plot(epochs, val_auc, "b", label = "Validation auc")
ax1.set_title("Training and validation auc")
ax1.legend()

ax2.plot(epochs, loss, "bo", label = "Training loss", color = 'red')
ax2.plot(epochs, val_loss, "b", label = "Validation loss", color = 'red')
ax2.set_title("Training and validation loss")
ax2.legend()

plt.show()

In [ ]:
# save model
model_EfficientNetB7.save('./EfficientNetB7_TPU.h5')

<a id="5"></a>
<h1 style='color:#0A0502; background:white; border:0'><center>Visualization of CNN intermediate activations</center></h1>

[**Back to the table of contents**](#start)

In [ ]:
def activation_layer_vis(img, activation_layer = 0, layers = 10):
    layer_outputs = [layer.output for layer in model_EfficientNetB7.layers[:layers]]
    activation_model = models.Model(inputs = model_EfficientNetB7.input, outputs = layer_outputs)
    activations = activation_model.predict(img)
    
    rows = int(activations[activation_layer].shape[3] / 3)
    cols = int(activations[activation_layer].shape[3] / rows)
    fig, axes = plt.subplots(rows, cols, figsize = (15, 15 * cols))
    axes = axes.flatten()
    
    for i, ax in zip(range(activations[activation_layer].shape[3]), axes):
        ax.matshow(activations[activation_layer][0, :, :, i], cmap = 'viridis')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
img_tensor = build_dataset(
    pd.Series(train_img[0]), bsize = 1,repeat = False, 
    shuffle = False, augment = False, cache = False)

In [ ]:
activation_layer_vis(img_tensor)

In [ ]:
def all_activations_vis(img, layers = 10):
    layer_outputs = [layer.output for layer in model_EfficientNetB7.layers[:layers]]
    activation_model = models.Model(inputs = model_EfficientNetB7.input, outputs = layer_outputs)
    activations = activation_model.predict(img)
    
    layer_names = []
    for layer in model_EfficientNetB7.layers[:layers]: 
        layer_names.append(layer.name) 

    images_per_row = 3
    for layer_name, layer_activation in zip(layer_names, activations): 
        n_features = layer_activation.shape[-1] 

        size = layer_activation.shape[1] 

        n_cols = n_features // images_per_row 
        display_grid = np.zeros((size * n_cols, images_per_row * size)) 

        for col in range(n_cols): 
            for row in range(images_per_row): 
                channel_image = layer_activation[0, :, :, col * images_per_row + row] 
                channel_image -= channel_image.mean() 
                channel_image /= channel_image.std() 
                channel_image *= 64 
                channel_image += 128 
                channel_image = np.clip(channel_image, 0, 255).astype('uint8') 
                display_grid[col * size : (col + 1) * size, 
                             row * size : (row + 1) * size] = channel_image 
        scale = 1. / size 
        plt.figure(figsize=(scale * 5 * display_grid.shape[1], 
                            scale * 5 * display_grid.shape[0])) 
        plt.title(layer_name) 
        plt.grid(False)
        plt.axis('off')
        plt.imshow(display_grid, aspect = 'auto', cmap = 'viridis')

In [ ]:
all_activations_vis(img_tensor, 3)

<a id="6"></a>
<h1 style='color:#0A0502; background:white; border:0'><center>Prediction</center></h1>

[**Back to the table of contents**](#start)

For submission you need to create a new notebook with GPU and load the pretrained weights from this notebook. 

In [ ]:
test_df = build_dataset(
    test_images, bsize = BATCH_SIZE, repeat = False, 
    shuffle = False, augment = False, cache = False)

In [ ]:
ss[label_cols] = model_EfficientNetB7.predict(test_df)
ss.to_csv('submission.csv', index = False)
ss.head()